# Creating a Job Description Dataset

## Importing our Greenhouse API Keys & Declarations

In [1]:
import os
import re
import ast
import json
import requests
import base64
import pandas as pd
from dotenv import load_dotenv

# Load API key (same as working test script)
api_key = os.getenv('GREENHOUSE_API_KEY')

if not api_key:
    print("❌ No API key found!")
else:
    print("✅ API key loaded successfully")

✅ API key loaded successfully


In [2]:
# Get Greenhouse jobs and create DataFrame (exact same method that worked)
credentials = base64.b64encode(f"{api_key}:".encode()).decode()
headers = {"Authorization": f"Basic {credentials}"}

## Creating a Dataframe of all job postings

### The Dataframe

In [3]:
# Check if CSV exists first
csv_file = 'data/greenhouse_jobs.csv'

if os.path.exists(csv_file):
    # Load from CSV
    jobs_df = pd.read_csv(csv_file)
    print(f"✅ Loaded {len(jobs_df)} jobs from CSV")
else:
    # Make API call
    job_postings_params = {
        "per_page": 500,
        "page": 1,
        "full_content": True,
        "internal": False
    }
    
    job_postings = requests.get('https://harvest.greenhouse.io/v1/jobs',
                               headers=headers, params=job_postings_params)
    
    print(f"Status: {job_postings.status_code}")
    
    if job_postings.status_code == 200:
        jobs = job_postings.json()
        jobs_df = pd.DataFrame(jobs)
        
        # Ensure directory exists
        os.makedirs(os.path.dirname(csv_file), exist_ok=True)

        # Save to CSV for next time
        jobs_df.to_csv(csv_file, index=False)
        
        print(f"🎉 Success! Found {len(jobs_df)} jobs")
    else:
        print(f"❌ Error: {job_postings.status_code}")
        print(f"Response: {job_postings.text[:200]}")
        jobs_df = None

# Continue with your existing DataFrame operations
if jobs_df is not None:
    print(f"📊 DataFrame shape: {jobs_df.shape}")
    print(f"📋 Columns: {list(jobs_df.columns)}")
    print(jobs_df.head())

✅ Loaded 246 jobs from CSV
📊 DataFrame shape: (246, 18)
📋 Columns: ['id', 'name', 'requisition_id', 'notes', 'confidential', 'is_template', 'copied_from_id', 'status', 'created_at', 'opened_at', 'closed_at', 'updated_at', 'departments', 'offices', 'hiring_team', 'openings', 'custom_fields', 'keyed_custom_fields']
           id                     name requisition_id  notes  confidential  \
0  4002141005         Senior Recruiter              1    NaN         False   
1  4002859005  Recruiting Coordinator               2    NaN         False   
2  4002971005  Enterprise Partnerships              4    NaN         False   
3  4002972005           Community Lead              5    NaN         False   
4  4003052005        Technical Writer               6    NaN         False   

   is_template  copied_from_id  status                created_at  \
0        False             NaN  closed  2022-02-23T16:58:29.538Z   
1        False    4.002141e+09  closed  2022-03-01T20:51:57.710Z   
2        Fal

### If importing from CSV, run code below
- Parses complex nested columns back from strings

In [4]:
# (Only run this if you need to work with departments, offices, etc. as actual lists/dicts)

def safe_eval(x):
    """Safely convert string representation back to Python object"""
    if pd.isna(x) or x == 'nan':
        return None
    try:
        return ast.literal_eval(x)
    except:
        return x

# Parse complex columns back to their original structure
complex_columns = ['departments', 'offices', 'hiring_team', 'openings', 'custom_fields', 'keyed_custom_fields']

for col in complex_columns:
    if col in jobs_df.columns:
        jobs_df[col] = jobs_df[col].apply(safe_eval)
        print(f"✅ Parsed {col}")

print("\nNow departments is a proper list:")
print(f"Type: {type(jobs_df.iloc[0]['departments'])}")
if jobs_df.iloc[0]['departments']:
    print(f"First department: {jobs_df.iloc[0]['departments'][0]}")


✅ Parsed departments
✅ Parsed offices
✅ Parsed hiring_team
✅ Parsed openings
✅ Parsed custom_fields
✅ Parsed keyed_custom_fields

Now departments is a proper list:
Type: <class 'list'>
First department: {'id': 4002059005, 'name': 'People', 'parent_id': None, 'parent_department_external_id': None, 'child_ids': [], 'child_department_external_ids': [], 'external_id': None}


### Figuring out Unique Departments

In [5]:
from collections import Counter
# Extract all department names from the departments column
all_departments = []

for dept_list in jobs_df['departments']:
    if dept_list:  # Check if the list is not empty
        for dept in dept_list:
            if 'name' in dept:
                all_departments.append(dept['name'])

# Count frequencies
department_counts = Counter(all_departments)

# Convert to a DataFrame
departments_df = pd.DataFrame(department_counts.items(), columns=['department_name', 'job_count'])

# Optional: sort by job count descending
departments_df = departments_df.sort_values(by='job_count', ascending=False).reset_index(drop=True)

total_jobs = len(jobs_df)
print(f"Total jobs: {total_jobs}")
print(departments_df)

Total jobs: 246
                   department_name  job_count
0                      Engineering         87
1   Business Operations & Strategy         26
2                           People         25
3               Product Management         21
4              Customer Experience         21
5                   Legal & Policy         17
6             Business Development         13
7                   Communications         12
8                         Research          9
9                        Marketing          6
10                    Data Science          4
11                          Design          3
12                            Test          2


### Indexing for 'real' roles

In [18]:
# TODO

## Pulling the Job Description from Job_Posts

In [10]:
# Check if job posts CSV exists first
posts_csv_file = 'data/all_greenhouse_job_posts.csv'

if os.path.exists(posts_csv_file):
    # Load from CSV
    postings_df = pd.read_csv(posts_csv_file)
    print(f"✅ Loaded {len(postings_df)} job posts from CSV")
else:
    # Make API call
    post_params = {
        "per_page": 500,
        "page": 1
        # "active": True,  # Omit this to get all job posts
        # "full_content": True # This is the full job description
    }
    # GET: List Job Posts 
    job_posts = requests.get('https://harvest.greenhouse.io/v1/job_posts',
                            headers=headers, params=post_params)
    
    print(f"Status: {job_posts.status_code}")
    
    if job_posts.status_code == 200:
        postings = job_posts.json()
        postings_df = pd.DataFrame(postings)
        
        # Save to CSV for next time
        postings_df.to_csv(posts_csv_file, index=False)
        
        print(f"🎉 Success! Found {len(postings_df)} job posts")
    else:
        print(f"❌ Error: {job_posts.status_code}")
        print(f"Response: {job_posts.text[:200]}")
        postings_df = None

# Continue with your DataFrame operations
if postings_df is not None:
    print(f"📊 DataFrame shape: {postings_df.shape}")
    print(f"📋 Columns: {list(postings_df.columns)}")

Status: 200
🎉 Success! Found 401 job posts
📊 DataFrame shape: (401, 15)
📋 Columns: ['id', 'active', 'live', 'first_published_at', 'title', 'location', 'internal', 'external', 'job_id', 'content', 'internal_content', 'updated_at', 'created_at', 'demographic_question_set_id', 'questions']


### Filtering by 'content' for JD

In [7]:
# Get the job description
job_description = postings_df['content']

# Method 1: See specific job descriptions (best for reading)
print("=== FIRST JOB DESCRIPTION ===")
print(job_description.iloc[0])  # First job
print("\n" + "="*50 + "\n")

=== FIRST JOB DESCRIPTION ===
<p>As we enter into this next phase of growth, we are looking to more than double our team and implement creative talent processes and strategies that can preserve and grow our culture of innovation. We are looking to make our first talent acquisition hire to help us scale and build out a world-class team. While the majority of our hiring will be for technical roles, we are looking for a technical recruiter to help us build a leading and diverse engineering team.</p>
<h3>Responsibilities:</h3>
<ul>
<li>Develop overall talent acquisition strategy</li>
<li>Build out processes and tools necessary to manage TA at scale</li>
<li>End-to-end recruitment from sourcing through to close</li>
<li>Understand business goal and manage internal stakeholders to drive outcomes</li>
<li>Partner with hiring managers to develop and execute systematic hiring process across all searches</li>
<li>Develop and maintain an excellent candidate experience</li>
<li>Identify, source, a

#### Cleaning the JD
- Removing html headings

In [8]:
# import re

def clean_html(text):
    """Remove HTML tags from text"""
    if pd.isna(text):
        return ""
    # Remove HTML tags
    clean = re.sub('<.*?>', '', str(text))
    # Replace common HTML entities
    clean = clean.replace('&nbsp;', ' ').replace('&amp;', '&')
    return clean.strip()

print("\n=== CLEAN TEXT DESCRIPTIONS ===")
# for i in range(min(3, len(postings_df))):
title = postings_df.iloc[0].get('title', 'No title')
content = clean_html(postings_df.iloc[0]['content'])
    
print(f"\nTitle: {title}")
print(f"Clean Description: {content}")
print("="*50)


=== CLEAN TEXT DESCRIPTIONS ===

Title: Senior Recruiter
Clean Description: As we enter into this next phase of growth, we are looking to more than double our team and implement creative talent processes and strategies that can preserve and grow our culture of innovation. We are looking to make our first talent acquisition hire to help us scale and build out a world-class team. While the majority of our hiring will be for technical roles, we are looking for a technical recruiter to help us build a leading and diverse engineering team.
Responsibilities:

Develop overall talent acquisition strategy
Build out processes and tools necessary to manage TA at scale
End-to-end recruitment from sourcing through to close
Understand business goal and manage internal stakeholders to drive outcomes
Partner with hiring managers to develop and execute systematic hiring process across all searches
Develop and maintain an excellent candidate experience
Identify, source, and interview candidates
Manage 

## Creating a condensed DF of job id, title, department, & JD

In [12]:
# Check if the cleaned_job_dataset.csv exists

if os.path.exists('data/cleaned_job_dataset.csv'):
    print("✅ Cleaned dataset already exists, loading from file...")
    job_dataset = pd.read_csv('data/cleaned_job_dataset.csv')
    print(f"📊 Loaded {len(job_dataset)} job postings")
    print(f"📈 Departments: {job_dataset['job_department'].value_counts().head()}")

else:
    print("🔄 Creating cleaned dataset...")

    # Load the data
    jobs_df = pd.read_csv('data/greenhouse_jobs.csv')
    posts_df = pd.read_csv('data/all_greenhouse_job_posts.csv')

    # Extract department name from departments field
    def get_department_name(dept_str):
        if pd.isna(dept_str):
            return 'Unknown'
        try:
            dept_list = ast.literal_eval(dept_str)
            return dept_list[0]['name'] if dept_list else 'Unknown'
        except:
            return 'Unknown'

    jobs_df['department_name'] = jobs_df['departments'].apply(get_department_name)

    # DEDUPLICATE: Keep only one post per job_id (prefer external, then active posts)
    print(f"📊 Before deduplication: {len(posts_df)} job posts")
    print(f"📊 Unique job_ids: {posts_df['job_id'].nunique()}")
    
    # Sort by preference: external=True, active=True, then by id (newest first)
    posts_df_sorted = posts_df.sort_values(['job_id', 'external', 'active', 'id'], 
                                          ascending=[True, False, False, False])
    
    # Keep only the first (best) post per job_id
    posts_df_unique = posts_df_sorted.drop_duplicates(subset=['job_id'], keep='first')
    
    print(f"📊 After deduplication: {len(posts_df_unique)} job posts")

    # Merge dataframes
    merged_df = posts_df_unique.merge(jobs_df[['id', 'department_name']], 
                                     left_on='job_id', 
                                     right_on='id', 
                                     how='inner')

    # Create final dataframe
    job_dataset = merged_df[['job_id', 'department_name', 'title', 'content']].rename(columns={
        'job_id': 'job_id',
        'department_name': 'job_department', 
        'title': 'job_title',
        'content': 'job_description'
    })

    # Save to CSV
    job_dataset.to_csv('data/cleaned_job_dataset.csv', index=False)

    print(f"✅ Created dataset with {len(job_dataset)} job postings")
    print(f"📊 Columns: {list(job_dataset.columns)}")
    print(f"📈 Departments: {job_dataset['job_department'].value_counts().head()}") 

🔄 Creating cleaned dataset...
📊 Before deduplication: 401 job posts
📊 Unique job_ids: 236
📊 After deduplication: 236 job posts
✅ Created dataset with 236 job postings
📊 Columns: ['job_id', 'job_department', 'job_title', 'job_description']
📈 Departments: job_department
Engineering                       82
People                            25
Business Operations & Strategy    25
Product Management                20
Customer Experience               19
Name: count, dtype: int64
